# Init

In [3]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor

os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [4]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [5]:
conn=init_connection(dsn)

2021-08-18 17:57:04.780 INFO    root: Connecting to the PostgreSQL database...
2021-08-18 17:57:04.792 INFO    root: PostgreSQL database version: PostgreSQL 13.4 (Ubuntu 13.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-08-18 17:57:04.793 INFO    root: PostgreSQL connection status: 0
2021-08-18 17:57:04.794 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


In [6]:
from typing import Union,List,Dict,Optional
import pandas as pd
def create_dataframe(data: Union[List, Dict, pd.Series], column_names: List = None, sort: bool = False, sort_by: Optional[str] = None, asc:bool=True,date_time_format: bool = False) -> pd.DataFrame:
    if data:

        df = pd.DataFrame(data, columns=column_names)
        df.index.name = ('No.')
        if date_time_format:
            df['Date/Time'] = pd.to_datetime(df['Date/Time'],
                                             format='%Y-%m-%d %H:%M:%S')

            # df.sort_values(by=['Date/Time'], inplace=True,
            #                ascending=False, ignore_index=True)
        if sort:

            df.sort_values(by=[sort_by], inplace=True,
                           ascending=asc, ignore_index=True)

            # dfStyler = df.style.set_properties(**{'text-align': 'center'})
            # dfStyler.set_table_styles(
            #     [dict(selector='th', props=[('text-align', 'center')])])

        return df

# 9/8/2021

In [5]:
query_sql="""
 SELECT
                                        p.id as "ID",
                                        p.name as "Name",
                                        description as "Description",
                                        dt.name as "Deployment Type",
                                        p.updated_at as "Date/Time"
                                        
                                    FROM
                                        public.project p
                                        LEFT JOIN deployment_type dt ON dt.id = p.deployment_id;
                                    """

In [20]:
query_sql="""SELECT
                                editor_config
                            FROM
                                public.editor
                            WHERE
                                project_id = 1;"""

In [14]:
query,column_names=db_fetchall(query_sql,conn,fetch_col_name=True,return_dict=True)



In [ ]:
query

In [ ]:
[dict(row) for row in query]

In [ ]:
selection=[x.ID for x in query if x.Name=="My Third Dataset"][0]
selection

3

In [14]:
dataset_tmp = []
if query:
    for dataset in query:

        converted_datetime = dataset.Date_Time.strftime(
            '%Y-%m-%d %H:%M:%S')

        # convert datetime with TZ to (2021-07-30 12:12:12) format
        dataset = dataset._replace(
            Date_Time=converted_datetime)
        dataset_tmp.append(dataset)

In [15]:
df=create_dataframe(dataset_tmp,column_names=column_names,sort=True,sort_by='ID',asc=True,date_time_format=True)

In [16]:
df_loc=df.loc[:,"ID":"Date/Time"]
df_loc
df

,ID,Name,Description,Deployment Type,Date/Time
0,2,First Project,This project is for object detection,Object Detection with Bounding Boxes,2021-07-21 10:01:09
1,3,Second Project,Second project entry test\n,Object Detection with Bounding Boxes,2021-07-22 23:19:30
2,4,My Third Project,Third project entry,Image Classification,2021-07-22 23:53:46
3,5,My Fourth Project,Fourth project entry,Object Detection with Bounding Boxes,2021-07-23 00:06:10
4,6,My Fifth Project,Fifth project entry,Object Detection with Bounding Boxes,2021-07-23 00:28:35
5,7,My Sixth Project,Sixth project entry,Object Detection with Bounding Boxes,2021-07-23 00:39:59
6,8,My,None,Image Classification,2021-08-12 13:32:00
7,12,My Sixth Projec,None,None,2021-08-12 13:34:03
8,24,My Seventh Project,Some seventh project entry\n,Image Classification,2021-08-12 14:24:56
9,29,My Eight Project,Eight Project Entry,Object Detection with Bounding Boxes,2021-08-17 00:44:27


In [ ]:
df_dict=df.to_dict(orient='index')

In [ ]:
if df_dict.get(3):
    print("yes")
else:
    print("no")

no


In [ ]:
name=(df_loc[df_loc["ID"]==3].Name)[1]
name

'My Third Dataset'

In [ ]:
df_loc.loc[df_loc["ID"]==3,'Name'].values[0]

'My Third Dataset'

In [ ]:
some_dict={'one':[{'name':'hello','info':'sadfa'},{'name':'test','info':'jvhjfgh'}]}

In [ ]:
multi_dict={}
for outerkey,innerDict in some_dict.items():
    for innerkey,value in some_dict.items():
        multi_dict[(outerkey,innerkey)]=value

In [ ]:
pd.DataFrame.from_dict(multi_dict)

,one
,one
0,"{'name': 'hello', 'info': 'sadfa'}"
1,"{'name': 'test', 'info': 'jvhjfgh'}"


In [ ]:
test=[1,2,3,4,5]
for z in zip(test[:],test[1:]):
    # if x==y:
    #     pass
    # else:
    #     print("Not equal")
    for i in z:
        print(i)

In [ ]:
# MIME: type/subtype
#get filetype 
import mimetypes
from pathlib import Path
from streamlit.uploaded_file_manager import UploadedFile
filepath=Path('/home/rchuzh/Documents/aruco/train/IMG_20210315_181136.jpg'
)
def get_filetype(file:Union[str,Path,UploadedFile]):
    if isinstance(file,(str,Path)):
        mime_type=mimetypes.guess_type(file)[0]
        filetype=str(Path(mime_type).parent)
    elif isinstance(file,UploadedFile):
        mime_type=file.type
        
    filetype=str(Path(mime_type).parent)
    return filetype

In [ ]:
file_tuple=[Path('/home/rchuzh/Documents/aruco/train/IMG_20210315_181136.jpg'
),Path('/home/rchuzh/Documents/aruco/test/IMG_20210315_181130.jpg')]
filetype1,filetype2=list(map(get_filetype,file_tuple))

NameError: name 'get_filetype' is not defined

In [ ]:
filetype1,filetype2

('image', 'image')

In [ ]:
# compare 2 values
def compare_filetypes(file_tuple:tuple):
    filetype1,filetype2=list(map(get_filetype,file_tuple))
    
    print(file_tuple)
    if filetype1==filetype2:
        return True
    else:
        
        return False

In [ ]:
uploaded_files=[1,2,3,4,4]


filetypes=map(compare_filetypes,zip(uploaded_files[:],uploaded_files[1:]))
for filetype in filetypes:
    if filetype:
        log_info("Filetype passed")
        pass
    else:
        log_error("Filetype different")
        break

2021-08-04 09:20:29.278 ERROR   root: Filetype different


(1, 2)


In [ ]:
# fileuploader consist of more tha one file upload
# if one file then no execution of callback
def check_filetype_category(uploaded_files:Union[str,Path,UploadedFile]):
    if uploaded_files:
        if len(uploaded_files)==1:
            filetype=get_filetype(uploaded_files)

        else:
            filetypes=map(compare_filetypes,zip(uploaded_files[:],uploaded_files[1:]))
            for filetype in filetypes:
                if filetype:
                    log_info("Filetype passed")
                    pass
                else:
                    log_error("Filetype different")
                    break



SyntaxError: unexpected EOF while parsing (<ipython-input-18-d4711ecc504b>, line 4)

In [ ]:
numbers1=[1,2,3,4]
numbers2=[5,6,7,8]
number=Path('3')
result=map(lambda x,y:x+y,numbers1,numbers2) #iterator
result=list(result)
result
isinstance(number,Path)

True

In [ ]:
number=[3]
len(number)

1

In [7]:
import shutil
from pathlib import Path
from core.utils.file_handler import delete_file_directory

In [8]:
test_file='readme.txt'
directory='/home/rchuzh/Documents'
filepath=Path(directory)/test_file
filepath

PosixPath('/home/rchuzh/Documents/readme.txt')

In [9]:
delete_file_directory(filepath)

2021-08-09 10:58:35.358 INFO    root: /home/rchuzh/Documents/readme.txt
2021-08-09 10:58:35.362 ERROR   root: Invalid path given......


In [12]:
random_folder='some_random_folder'
folder_path=Path(directory)/random_folder
folder_path

PosixPath('/home/rchuzh/Documents/some_random_folder')

In [13]:
delete_file_directory(folder_path)

2021-08-09 10:59:01.548 INFO    root: /home/rchuzh/Documents/some_random_folder
2021-08-09 10:59:01.554 INFO    root: Successfully deleted some_random_folder folder


## Check if file exist in dataset

In [5]:
dataset_path=Path("/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset")
new_data_path=Path("/home/rchuzh/Documents/aruco/train/*.jpg")

In [6]:
from time import perf_counter
from timeit import timeit
perf_metrics=[]

In [27]:
from glob import iglob
new_data_list_name=[Path(x).name for x in iglob(str(new_data_path))]
len(new_data_list_name)
iglob_time=timeit("""[Path(x).name for x in iglob(str(new_data_path))]""",setup="from glob import iglob;from pathlib import Path; new_data_path=Path('/home/rchuzh/Documents/aruco/train/*.jpg')",number=10)
iglob_time

0.039029192994348705

In [ ]:
def check_if_exist(dataset_path):
    exist=[]
    for new_data in new_data_list_name:
        _filepath=dataset_path/new_data
        if _filepath.is_file():
            exist.append(new_data)
        else:
            log_info(f"{new_data} is NEW!!!!")




In [33]:
list(set([1,2,3,4,4,4,3,4]))


[1, 2, 3, 4]

In [35]:
list(set([1,2,3,4,4,4,3,4]+[5,6,7,8,1,2,5,5,6,8,8]))

[1, 2, 3, 4, 5, 6, 7, 8]

In [36]:
from tqdm import tqdm
from time import sleep
x=list(range(1000000))
for i in tqdm(x,desc='test',unit='test',ascii ="123456789$"):
    sleep(0)
    pass

test: 100%|$$$$$$$$$$| 1000000/1000000 [00:02<00:00, 355570.33test/s]


In [2]:
x,y=(None,None)

In [23]:
from enum import IntEnum,Enum
class DataPermission(IntEnum):
    ViewOnly = 0
    Append = 1

    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return DataPermission[s]
        except KeyError:
            raise ValueError()


In [24]:
DataPermission.from_string('ViewOnly')

<DataPermission.ViewOnly: 0>

# 11/8/2021

In [7]:
from appdirs import *

In [8]:
appname="SuperApp"
appauthor="shrdc"
user_log_dir(appname,appauthor)

'/home/rchuzh/.cache/SuperApp/log'

In [15]:
from core.utils.file_handler import get_config_dir,get_data_dir

dataset_name='My Second Dataset'
BASE_DATA_DIR = Path(get_data_dir())
MEDIA_ROOT = BASE_DATA_DIR / 'app_media'
DATASET_DIR = MEDIA_ROOT / 'dataset'
PRE_TRAINED_MODEL_DIR = MEDIA_ROOT / 'pre-trained-models'
PROJECT_DIR = MEDIA_ROOT / 'project'

In [12]:
print(BASE_DATA_DIR,MEDIA_ROOT,DATASET_DIR,PRE_TRAINED_MODEL_DIR,PROJECT_DIR)

/home/rchuzh/.local/share/integrated-vision-inspection-system /home/rchuzh/.local/share/integrated-vision-inspection-system/app_media /home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset /home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/pre-trained-models /home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project


In [22]:
dataset_path="/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/my-fifth-dataset"
dataset_path = Path(dataset_path) / "./*"
dataset_path

PosixPath('/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/my-second-dataset/*')

In [23]:
from glob import iglob
for data_path in iglob(str(dataset_path)):
    print(data_path)

In [17]:
from PIL import Image
image_path="/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/my-fifth-dataset/house.jpg"

In [18]:
with Image.open(Path(image_path)) as img:
    image=img

In [22]:
Path(image.filename).name

'house.jpg'

In [34]:
from videoprops import get_video_properties,get_audio_properties
video_path="/home/rchuzh/Videos/obs/2021-08-11_13-59-06.mp4"
audio_path="/home/rchuzh/Music/bp.mp3"

Path(audio_path).is_file()


True

In [21]:
props=get_video_properties(video_path)
display(props)

{'index': 0,
 'codec_name': 'h264',
 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10',
 'profile': 'High',
 'codec_type': 'video',
 'codec_time_base': '1/120',
 'codec_tag_string': 'avc1',
 'codec_tag': '0x31637661',
 'width': 1920,
 'height': 1080,
 'coded_width': 1920,
 'coded_height': 1088,
 'has_b_frames': 2,
 'sample_aspect_ratio': '1:1',
 'display_aspect_ratio': '16:9',
 'pix_fmt': 'yuv420p',
 'level': 42,
 'color_range': 'tv',
 'color_space': 'bt709',
 'color_transfer': 'bt709',
 'color_primaries': 'bt709',
 'chroma_location': 'left',
 'refs': 1,
 'is_avc': 'true',
 'nal_length_size': '4',
 'r_frame_rate': '60/1',
 'avg_frame_rate': '60/1',
 'time_base': '1/15360',
 'start_pts': 0,
 'start_time': '0.000000',
 'duration_ts': 22016,
 'duration': '1.433333',
 'bit_rate': '480591',
 'bits_per_raw_sample': '8',
 'nb_frames': '86',
 'disposition': {'default': 1,
  'dub': 0,
  'original': 0,
  'comment': 0,
  'lyrics': 0,
  'karaoke': 0,
  'forced': 0,
  'hearing_impaired

In [27]:
display_attributes=f"""
                ### **DATA**
            
                ___
                ### **MEDIA ATTRIBUTES**
                - Codec:{props['codec_name']}
                - Width: {props['width']}
                - Height: {props['height']}
                - Duration: {float(props['duration']):.2f}s
                - Frame rate: {props['avg_frame_rate']}
                - Frame count: {props['nb_frames']}
                
                """
display_attributes

'\n                ### **DATA**\n            \n                ___\n                ### **MEDIA ATTRIBUTES**\n                - Codec:h264\n                - Width: 1920\n                - Height: 1080\n                - Duration: 1.43s\n                - Frame rate: 60/1\n                - Frame count: 86\n                \n                '

In [36]:
props=get_audio_properties(audio_path)
props

{'index': 0,
 'codec_name': 'mp3',
 'codec_long_name': 'MP3 (MPEG audio layer 3)',
 'codec_type': 'audio',
 'codec_time_base': '1/44100',
 'codec_tag_string': '[0][0][0][0]',
 'codec_tag': '0x0000',
 'sample_fmt': 'fltp',
 'sample_rate': '44100',
 'channels': 2,
 'channel_layout': 'stereo',
 'bits_per_sample': 0,
 'r_frame_rate': '0/0',
 'avg_frame_rate': '0/0',
 'time_base': '1/14112000',
 'start_pts': 0,
 'start_time': '0.000000',
 'duration_ts': 2553983057,
 'duration': '180.979525',
 'bit_rate': '320000',
 'disposition': {'default': 0,
  'dub': 0,
  'original': 0,
  'comment': 0,
  'lyrics': 0,
  'karaoke': 0,
  'forced': 0,
  'hearing_impaired': 0,
  'visual_impaired': 0,
  'clean_effects': 0,
  'attached_pic': 0,
  'timed_thumbnails': 0}}

In [43]:
display_attributes=f"""
                ### **DATA**
              
                ___
                ### **MEDIA ATTRIBUTES**
                - Codec:{props['codec_long_name']}
                - Channel Layout: {props['channel_layout']}
                - Channels: {props['channels']}
                - Duration: {float(props['duration']):.2f}s
                - Bit rate: {(float(props['bit_rate'])/1000)}kbps
                - Sampling rate: {(float(props['sample_rate'])/1000):.2f}kHz
            
                """

In [38]:
from pprint import PrettyPrinter
pp=PrettyPrinter(indent=2)

In [42]:
print(display_attributes)


                ### **DATA**
              
                ___
                ### **MEDIA ATTRIBUTES**
                - Codec:MP3 (MPEG audio layer 3)
                - Channel Layout: stereo
                - Channels: 2
                - Duration: 181s
                - Bit rate: 320.0kbps
                - Sampling rate: 44.10kHz
            
                


In [1]:
# HEX to RGB

h = input('Enter hex: ').lstrip('#')
print('RGB =', tuple(int(h[i:i+2], 16) for i in (0, 2, 4)))

RGB = (41, 98, 255)


In [1]:
import colorsys
colorsys.rgb_to_hsv(41,98,255)

(0.6222741433021807, 0.8392156862745098, 255)

In [2]:
from typing import NamedTuple

class RGB(NamedTuple):
    R:int
    G:int
    B:int
class HSV(NamedTuple):
    H:int
    S:int
    V:int
def hex_to_hsv(hex_code):
    hex_value=hex_code.lstrip('#')
    rgb=RGB._make(tuple(int(hex_value[i:i+2], 16) for i in (0, 2, 4)))
    hsv=HSV._make((i for i in colorsys.rgb_to_hsv(rgb.R,rgb.G,rgb.B)))
    return hsv


In [3]:
hex_to_hsv('#80CBC4')

HSV(H=0.48444444444444446, S=0.3694581280788177, V=203)

In [4]:
rgb=RGB._make([1,2,3])
rgb

RGB(R=1, G=2, B=3)

In [6]:
from colorutils import Color
c=Color(hex='#80CBC4')

In [10]:
c.hsv
hsv=HSV._make(c.hsv)
hsv

HSV(H=174.4, S=0.369, V=0.796)

# 12/8/2021

In [9]:
from psycopg2 import sql
def check_if_exists(table: str, column_name: str, condition, conn):
    # Separate schema and tablename from 'table'
    schema, tablename = [i for i in table.split('.')]
    check_if_exists_SQL = sql.SQL("""
                        SELECT
                            EXISTS (
                                SELECT
                                    *
                                FROM
                                    {}
                                WHERE
                                    {} = %s);
                            """).format(sql.Identifier(schema, tablename), sql.Identifier(column_name))
    check_if_exists_vars = [condition]
    exist_flag = db_fetchone(check_if_exists_SQL, conn, check_if_exists_vars).exists

    return exist_flag

In [10]:
table='public.project'
column_name='name'
condition='My Sixth Project'
check_if_exists(table,column_name,condition,conn)

True

In [14]:
def random(**kwargs):
    for i in kwargs.items():
        print(i)
test={'a':1,'b':2}
random(**test)

('a', 1)
('b', 2)


In [15]:
x=[1,2,3,4]
not x

False

In [16]:
def uppercase_decorator(function):
    def wrapper():
        func=function()
        make_uppercase =func.upper()
        
        return make_uppercase
    return wrapper

In [17]:
def say_hi():
    return 'hello there'

decorate=uppercase_decorator(say_hi)
decorate()

'HELLO THERE'

In [18]:
@uppercase_decorator
def say_hi():
    return 'hello there'

In [19]:
say_hi()

'HELLO THERE'

In [20]:
def split_string(function):
    def wrapper():
        func=function()
        splitted_string=func.split()
        return splitted_string

    return wrapper

In [21]:
@split_string
@uppercase_decorator
def say_hi():
    return 'hello there'

In [23]:
say_hi()

['HELLO', 'THERE']

In [ ]:
from json import dumps
random_dict=[{'bbox':[1,2,3,4],'seg':[4,5,6,7]}]
random_json_array=[]
for i in random_dict:
    random_json_array.append(dumps(i))

In [7]:
random_json_array

['"bbox"', '"seg"']

In [1]:
from enum import IntEnum
class ProjectPagination(IntEnum):
    Dashboard = 0
    New = 1
    Existing = 2
    NewDataset = 3

In [7]:
ProjectPagination.Existing+1==ProjectPagination.NewDataset

True

In [12]:
class DeploymentType(IntEnum):
    Image_Classification = 1
    OD = 2
    Instance = 3
    Semantic = 4

    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return DeploymentType[s]
        except KeyError:
            raise ValueError()

DEPLOYMENT_TYPE = {
    "Image Classification": DeploymentType.Image_Classification,
    "Object Detection with Bounding Boxes": DeploymentType.OD,
    "Semantic Segmentation with Polygons": DeploymentType.Instance,
    "Semantic Segmentation with Masks": DeploymentType.Semantic
}

In [13]:
DEPLOYMENT_TYPE["Object Detection with Bounding Boxes"]

<DeploymentType.OD: 2>

In [27]:
type(DEPLOYMENT_TYPE["Object Detection with Bounding Boxes"])

<enum 'DeploymentType'>

In [28]:
isinstance(DEPLOYMENT_TYPE["Object Detection with Bounding Boxes"],IntEnum)

True

In [1]:
[x for x in range(1)]

[0]

In [24]:
some_string="""sgfsdgsdfgsdfg   asdfasdfa asdfad    
asfsfas
asdfad


asdfadsf"""
print(some_string)
no_line_breaks=[x.strip() for x in some_string.split("\n") if x.strip()]
# for x in some_string.split("\n"):

# [x.strip() for x in some_string.split("\n")]
no_line_breaks=some_string.replace("\n"," ")
no_line_breaks
# " ".join([x.strip() for x in no_line_breaks])
some_string.split()
" ".join([x for x in some_string.split()])

sgfsdgsdfgsdfg   asdfasdfa asdfad    
asfsfas
asdfad


asdfadsf


'sgfsdgsdfgsdfg asdfasdfa asdfad asfsfas asdfad asdfadsf'

In [20]:
def myFun(*args):
    for arg in args:
        print(arg)

In [23]:
myFun('Hello',"Bye")

Hello
Bye


In [27]:
x=('Hello',"Bye")
myFun(*x)

Hello
Bye


In [12]:
def callback(x=None,y=None,z=None):
    # print(x,y,z)
    print("Hi")

In [16]:
x=(1,2,3,)
y={1:1,2:2,3:3}
from inspect import signature
def check_args_kwargs(wildcards:Union[List,Dict]):
    
    assert len(wildcards)==len(signature(callback).parameters),"Length of wildcards does not meet length of arguments required by callback function"

from typing import Callable,Any
def main_func(label:str,key:str=None,on_change:Callable[...,Any]=None,args:tuple=None,kwargs:dict=None):

    
    if on_change:
        wildcard=args if args else kwargs
        if args or kwargs:
            check_args_kwargs(wildcards=wildcard)
        if args:
            on_change(*args)
        elif kwargs:
            on_change(*kwargs)
        else:
            on_change()


In [17]:
main_func(label="test",key="test",on_change=callback)

TypeError: callback() argument after * must be an iterable, not NoneType

In [1]:
s=None

In [2]:
name=s if s is not None else 'f'

In [3]:
name